In [2]:
from get_features import get_dataframes
dataframe = get_dataframes()
print(get_dataframes.__doc__)
help(get_dataframes)




    gets cells that requires recontruction and matches specimen id and returns the cells ephys features

    Args: 
        recon_bool (bool), optional: determines whether cell should require reconstruction

    Returns:
        Dataframe: dataframe of cells and their features
    
Help on function get_dataframes in module get_features:

get_dataframes(recon_bool=True)
    gets cells that requires recontruction and matches specimen id and returns the cells ephys features
    
    Args: 
        recon_bool (bool), optional: determines whether cell should require reconstruction
    
    Returns:
        Dataframe: dataframe of cells and their features



In [3]:
import pandas as pd
from allensdk.core.cell_types_cache import CellTypesCache
ctc = CellTypesCache(manifest_file='cell_types/manifest.json')

cells = ctc.get_cells(require_reconstruction = True)

cell_index = { c['id']: c for c in cells }
print(cell_index[485909730].keys())
print(cell_index[485909730])
label_features = ["id", "species", "structure_layer_name", "structure_area_abbrev", "dendrite_type"]

all_ids = [[item[feature] for feature in label_features] for item in cells]
print(all_ids)

dataframe_labels = pd.DataFrame(all_ids, columns = label_features)
#print(dataframe_labels)

filtered_dataframe_labels = dataframe_labels[dataframe_labels["dendrite_type"].isin(["spiny", "aspiny"])]
print(filtered_dataframe_labels)
print(filtered_dataframe_labels["dendrite_type"].unique())



dict_keys(['reporter_status', 'cell_soma_location', 'species', 'id', 'name', 'structure_layer_name', 'structure_area_id', 'structure_area_abbrev', 'transgenic_line', 'dendrite_type', 'apical', 'reconstruction_type', 'disease_state', 'donor_id', 'structure_hemisphere', 'normalized_depth'])
{'reporter_status': 'positive', 'cell_soma_location': [8881.0, 953.839501299405, 7768.22695782726], 'species': 'Mus musculus', 'id': 485909730, 'name': 'Cux2-CreERT2;Ai14-205530.03.02.01', 'structure_layer_name': '5', 'structure_area_id': 385, 'structure_area_abbrev': 'VISp', 'transgenic_line': 'Cux2-CreERT2', 'dendrite_type': 'spiny', 'apical': 'intact', 'reconstruction_type': 'dendrite-only', 'disease_state': '', 'donor_id': 485250100, 'structure_hemisphere': 'right', 'normalized_depth': 0.478343598387418}
[[485909730, 'Mus musculus', '5', 'VISp', 'spiny'], [323865917, 'Mus musculus', '5', 'VISp', 'spiny'], [476135066, 'Mus musculus', '4', 'VISp', 'spiny'], [502614426, 'Mus musculus', '4', 'VISpl', 

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(filtered_dataframe_labels['dendrite_type'])
print(le.classes_)
filtered_dataframe_labels['dendrite_type_number'] = le.transform(filtered_dataframe_labels['dendrite_type'])
print(filtered_dataframe_labels[["dendrite_type", "dendrite_type_number"]])

le.fit(filtered_dataframe_labels['structure_layer_name'])
print(le.classes_)
filtered_dataframe_labels['dendrite_type_number'] = le.transform(filtered_dataframe_labels['dendrite_type'])
print(filtered_dataframe_labels[["dendrite_type", "dendrite_type_number"]])

['aspiny' 'spiny']
    dendrite_type  dendrite_type_number
0           spiny                     1
1           spiny                     1
2           spiny                     1
3           spiny                     1
4           spiny                     1
..            ...                   ...
662        aspiny                     0
663        aspiny                     0
664         spiny                     1
665         spiny                     1
666        aspiny                     0

[618 rows x 2 columns]


C:\Users\lynnh\AppData\Local\Temp\ipykernel_4724\3464022644.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataframe_labels['dendrite_type_number'] = le.transform(filtered_dataframe_labels['dendrite_type'])


In [5]:
from sklearn.preprocessing import OneHotEncoder
oneHB_area = OneHotEncoder()
oneHB_area.fit(filtered_dataframe_labels["structure_area_abbrev"].unique().reshape(1, -1))

#transformed_data = oneHB_area.transform(filtered_dataframe_labels['structure_area_abbrev'].to_numpy().reshape(1, -1))
new_dataframe = pd.get_dummies(filtered_dataframe_labels, columns = ["structure_area_abbrev"])
#filtered_dataframe_labels['structure_area_abbrev_number'] = oneHB_area.transform(filtered_dataframe_labels['structure_area_abbrev'])
#print(filtered_dataframe_labels[["structure_area_abbrev", "structure_area_abbrev_number"]])

print(new_dataframe)

            id       species structure_layer_name dendrite_type  \
0    485909730  Mus musculus                    5         spiny   
1    323865917  Mus musculus                    5         spiny   
2    476135066  Mus musculus                    4         spiny   
3    502614426  Mus musculus                    4         spiny   
4    519832676  Homo Sapiens                    3         spiny   
..         ...           ...                  ...           ...   
662  561469082  Homo Sapiens                    5        aspiny   
663  554807924  Mus musculus                  2/3        aspiny   
664  512322162  Mus musculus                   6b         spiny   
665  601958555  Homo Sapiens                    4         spiny   
666  464212183  Mus musculus                    5        aspiny   

     dendrite_type_number  structure_area_abbrev_AnG  \
0                       1                          0   
1                       1                          0   
2                       1  

In [55]:
Y = filtered_dataframe_labels['dendrite_type_number']
X = filtered_dataframe_labels["structure_layer_name"]

# print(len(X), len(Y)) # same number of rows
print(filtered_dataframe_labels[0:5])
import numpy as np

X = X.to_numpy().reshape(-1, 1)
Y = Y.to_numpy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.20, random_state=42)
print(X_train)

from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

clf = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)
clf.fit(X_train, y_train)

from sklearn import tree
tree.plot_tree(clf, proportion=True)
plt.show()

          id       species structure_layer_name structure_area_abbrev  \
0  485909730  Mus musculus                    5                  VISp   
1  323865917  Mus musculus                    5                  VISp   
2  476135066  Mus musculus                    4                  VISp   
3  502614426  Mus musculus                    4                 VISpl   
4  519832676  Homo Sapiens                    3                   MTG   

  dendrite_type  dendrite_type_number  
0         spiny                     1  
1         spiny                     1  
2         spiny                     1  
3         spiny                     1  
4         spiny                     1  
[['3']
 ['5']
 ['4']
 ['5']
 ['6a']
 ['5']
 ['5']
 ['5']
 ['5']
 ['2/3']
 ['6a']
 ['5']
 ['2/3']
 ['2/3']
 ['6b']
 ['5']
 ['6b']
 ['5']
 ['5']
 ['5']
 ['4']
 ['5']
 ['3']
 ['6a']
 ['4']
 ['5']
 ['3']
 ['5']
 ['2/3']
 ['6b']
 ['6a']
 ['5']
 ['4']
 ['4']
 ['6b']
 ['5']
 ['6']
 ['2']
 ['2/3']
 ['4']
 ['6b']
 ['2/3']
 ['2/3

ValueError: could not convert string to float: '6a'